### Nanocomp - 2025.02 - Prof. Omar
---
<p> 
Optimizing quantum wells for quantum bragg mirror detectors via genetic algorithms(NSGA-II).
</p>

- Guilherme Rigacci
- Marco Túlio Mata
- Lucas Gabriel

---

#### 1. _AUTOMATED RUN_

In [ ]:
%%capture

import include.run_prog as prog

ex1 = [4, 2e-9, 7e-9, 5e-9, 1, 2e-9, 7e-9]
param = ex1
if(len(param) != 7):
    raise ValueError("Parameter list must have exactly 7 elements.")
out_dir = prog.run_qbmd(param[0], param[1], param[2], param[3], param[4], param[5], param[6], create_param_folder=True)
print("Outputs in:", out_dir)

#### 2. _VISUALIZATION_

In [ ]:
import include.visualize as viz
out_dir = "/home/mark/nano.compQBMD/linux_executable/manual_runs/05x2.0_7.0__2.0__01x2.0_7.0/"
res = viz.visualize_all(out_dir, show=True)

### 3. _GENETIC ALGORITHM_

In [ ]:
import sys
from include.run_opt import run_optimization as run_ga

if __name__ == "__main__":
    try:
        print("Iniciando otimização GA multi-objetivo...")
        
        # Example for stating point
        initial_population_csv="../linux_executable/optimization_results/pareto_full_guilherme4320.csv"
        
        # You may want to clear any intermediate_sim_*.csv before running
        # results = run_ga() 
        results = run_ga(initial_population_csv)
        
        print("\n Otimização concluída com sucesso!")
        sys.exit(0)
    except KeyboardInterrupt:
        print("\n  Otimização interrompida pelo usuário")
        sys.exit(1)
    except Exception as e:
        print(f"\n:c Error durante a otimização: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)


### 4. _PARETO_ _FRONTS_

In [ ]:
# Load Pareto front results using pareto_extract module
from include.pareto_extract import load_pareto_results, display_best_solutions, visualize_pareto_2d, visualize_pareto_3d, get_statistics

# Load the most recent optimization results
try:
    df = load_pareto_results()
    print("\nColumns:", list(df.columns))
    print("\nFirst 5 solutions:")
    display(df.head())
except FileNotFoundError as e:
    print(e)
    df = None

In [ ]:
# Display summary statistics
if df is not None:
    stats = get_statistics(df)
    display(stats)

In [ ]:
# Display best solutions for each objective
if df is not None:
    display_best_solutions(df)

In [ ]:
# projections
if df is not None:
    visualize_pareto_2d(df, show=True, save=False)

In [ ]:
# representing 4 goals
if df is not None:
    visualize_pareto_3d(df, show=True, save=False)

### 5. _CONCLUSIONS_ - Parameter-Objective Relationships

In [ ]:
# Analyze relationships between input parameters and objectives
from include.pareto_extract import analyze_parameter_objective_relationships, print_parameter_analysis, get_conclusions_markdown

if df is not None:
    # Perform analysis on Pareto front data
    analysis = analyze_parameter_objective_relationships(df, top_percent=25)
    
    # Print detailed analysis
    print_parameter_analysis(analysis)

In [ ]:
# visualize correlacoes parametro-objetivo
import matplotlib.pyplot as plt
import numpy as np

if df is not None:
    param_cols = ['RW', 'RQWt', 'RQBt', 'MQWt', 'LW', 'LQWt', 'LQBt']
    obj_cols = ['max_energy_eV', 'max_photocurrent', 'quality_factor', 'prominence']
    
    # Calculate correlation matrix
    all_cols = param_cols + obj_cols
    corr_matrix = df[all_cols].corr()
    
    # Extract only parameter-objective correlations
    param_obj_corr = corr_matrix.loc[param_cols, obj_cols]
    
    # Create heatmap
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(param_obj_corr.values, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
    
    # Labels
    ax.set_xticks(range(len(obj_cols)))
    ax.set_xticklabels(['Energy', 'Photocurrent', 'Q-Factor', 'Prominence'], rotation=45, ha='right')
    ax.set_yticks(range(len(param_cols)))
    ax.set_yticklabels(param_cols)
    
    # Add correlation values as text
    for i in range(len(param_cols)):
        for j in range(len(obj_cols)):
            val = param_obj_corr.iloc[i, j]
            color = 'white' if abs(val) > 0.5 else 'black'
            ax.text(j, i, f'{val:.2f}', ha='center', va='center', color=color, fontsize=10)
    
    plt.colorbar(im, label='Correlation')
    plt.title('Parameter-Objective Correlation Matrix', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

### AI ANALYZE

In [ ]:
from IPython.display import Markdown, display

if df is not None:
    conclusions_md = get_conclusions_markdown(analysis)
    display(Markdown(conclusions_md))